In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

E:\anacoda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("../data/ccf_first_round_user_shop_behavior.csv")
shop = pd.read_csv("../data/ccf_first_round_shop_info.csv")
test = pd.read_csv("../data/evaluation_public.csv")

In [3]:
df = pd.merge(df, shop[["shop_id", "mall_id"]], how="left", on="shop_id")

In [4]:
df["time_stamp"] = pd.to_datetime(df['time_stamp'])

In [5]:
train = pd.concat([df,test])

In [6]:
mall_list = list(set(list(shop.mall_id)))

In [7]:
result = pd.DataFrame()

In [8]:
import math
def haversin(x):
    return (math.sin(x/2))**2

In [9]:
train.head()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,32.088040,122.308291,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,32.087970,122.308162,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,40.638214,117.365255,m_4079,NaN,s_181637,2017-08-02 13:10:00,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,31.197416,121.134451,m_6587,NaN,s_609470,2017-08-13 12:30:00,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,31.351320,122.255867,m_3005,NaN,s_3816766,2017-08-25 19:50:00,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [10]:
len(set(mall_list))

97

In [11]:
# step = 1
# for mall in mall_list:
#     print("目前正在跑第 %d 个mall, %d/79" % (step, step))
#     train1 = train[train.mall_id == mall].reset_index(drop=True)
#     train1["distance"]=0
#     EARTH_RADIUS = 6371.0
#     for index, row in train1.iterrows():
#         ################# 剔除异常值 （距离计算）###################
#         tmp_df = shop[(shop["shop_id"] == row["shop_id"]) & (shop["mall_id"] == row["mall_id"])]
#         distance = 0
#         if not tmp_df.empty:
#             for _, row2 in tmp_df.iterrows():
#                 vLon = row["longitude"] - row2["longitude"]
#                 vLat = row["latitude"] - row2["latitude"]
#                 distance = haversin(vLat) + math.cos(row["latitude"])*math.cos(row2["latitude"])*haversin(vLon)
#                 distance = 2 * EARTH_RADIUS * math.sin(math.sqrt(distance))
#                 train1.loc[train1.index==index, "distance"] = distance
#     train1.to_csv("../middlefile/train_v1.csv", index=False)
#     step += 1

In [12]:
weekday_weight = {
    0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 2, 7: 2
}
hour_weight = {
    0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1,
    6: 1, 7: 2, 8: 4, 9: 4, 10: 4, 11: 8,
    12: 8, 13: 8, 14: 4, 15: 4, 16: 4, 17: 8,
    18: 8, 19: 8, 20: 8, 21: 2, 22: 1, 23: 1
}

def get_date_and_time(train):
    train.loc[:, 'time'] = pd.to_datetime(train.time_stamp)
    train['weekday_time'] = train.time.dt.weekday + 1
    train['hour_time']  = train.time.dt.hour
    train['hour_weight'] = train.hour_time.map(hour_weight)
    train['weekday_weight'] = train.weekday_time.map(weekday_weight)
    return train.drop(['time'], axis=1)


In [13]:
# 过滤wifi  测试集合中没有在训练集合中出现的，就不用加进去了
# train_wifis = df["wifi_infos"].apply(lambda x: x.split(";")[0]).apply(lambda x: x.split("|")[0])
# test_wifis = test["wifi_infos"].apply(lambda x: x.split(";")[0]).apply(lambda x: x.split("|")[0])
train_wifi_set = df.wifi_infos.map(lambda x:[a.split('|')[0] for a in x.split(';')])
train_wifi_set = set([b for a in train_wifi_set for b in a])
test_wifi_set = test.wifi_infos.map(lambda x:[a.split('|')[0] for a in x.split(';')])
test_wifi_set = set([b for a in test_wifi_set for b in a])

In [14]:
public_wifi_set = set([a for a in train_wifi_set if a in test_wifi_set])

In [15]:
len(public_wifi_set)

121209

In [16]:
def get_distance(df):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-16-7fff03d04844>, line 2)

In [17]:
step=1
for mall in mall_list:
    print("################# ######目前正在跑第 %d 个mall, %d/97##########################" % (step, step))
    train1 = train[train.mall_id == mall].reset_index(drop=True)
#     train1["distance"] = 0
#     EARTH_RADIUS = 6371.0
#     for index, row in train1.iterrows():
#         ################# 剔除异常值 （距离计算）###################
#         tmp_df = shop[(shop["shop_id"] == row["shop_id"]) & (shop["mall_id"] == row["mall_id"])]
#         distance = 0
#         if not tmp_df.empty:
#             for _, row2 in tmp_df.iterrows():
#                 vLon = row["longitude"] - row2["longitude"]
#                 vLat = row["latitude"] - row2["latitude"]
#                 distance = haversin(vLat) + math.cos(row["latitude"])*math.cos(row2["latitude"])*haversin(vLon)
#                 distance = 2 * EARTH_RADIUS * math.sin(math.sqrt(distance))
#                 train1.loc[train1.index==index, "distance"] = round(distance*10, 3)
#         else:
#             train1.loc[train1.index==index, "distance"] = 999
#     train1 = train1[train1["distance"] < 100]
#     train1 = train1.drop(["distance"], axis=1)
    
    train1  = get_date_and_time(train1)
    
    l = []
    wifi_dict = {}
    for index, row in train1.iterrows():  
        r = {}
        wifi_list = [wifi.split('|') for wifi in row["wifi_infos"].split(";")]
        for i in wifi_list:
            if i[0] in public_wifi_set:
                # r = {"wifi_id" : wifi强度}
                r[i[0]] = int(i[1])
                # wifi出现次数
                if i[0] not in wifi_dict:
                    wifi_dict[i[0]] = 1
                else:
                    wifi_dict[i[0]] += 2
        l.append(r)
    delete_wifi = []
    # 剔除出现次数比较少的wifi，有可能是热点
    for i in wifi_dict:
        ### 这里可以重大改进
        if wifi_dict[i] < 18:
            delete_wifi.append(i)
    m = []
    for row in l:
        new = {}
        for n in row.keys():
            if n not in delete_wifi:
                new[n] = row[n]
        m.append(new)
    train1 = pd.concat([train1, pd.DataFrame(m)], axis=1)
    df_train = train1[train1.shop_id.notnull()]
    df_test = train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train["shop_id"].values))
    df_train['label'] = lbl.transform(list(df_train["shop_id"].values))
    num_class = df_train['label'].max() + 1
    params = {
        'objective' : 'multi:softmax',
        'eta': 0.1,
        'max_depth' : 9,
        'eval_metric' : 'merror',
        'seed' : 0,
        'missing' : -999,
        'num_class' : num_class,
        'silent' : 1
    }
    feature = [x for x in train1.columns if x not in ['user_id', 'label', 'shop_id', 'time_stamp', 'mall_id', 'wifi_infos']]
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain, 'train'), (xgbtrain, 'test')]
    num_rounds = 60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    df_test['label'] = model.predict(xgbtest)
    df_test['shop_id'] = df_test['label'].apply(lambda x : lbl.inverse_transform(int(x)))
    r = df_test[['row_id', 'shop_id']]
    result = pd.concat([result, r])
    result['row_id'] = result['row_id'].astype('int')
    result = result.drop_duplicates()
    result.to_csv("../result/sub.csv", index=False)
    step += 1
    print("============================ complete！ ====================================")

################# ######目前正在跑第 1 个mall, 1/97##########################


E:\anacoda\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.069325	test-merror:0.069325
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.058772	test-merror:0.058772
[2]	train-merror:0.052962	test-merror:0.052962
[3]	train-merror:0.049289	test-merror:0.049289
[4]	train-merror:0.045014	test-merror:0.045014
[5]	train-merror:0.041541	test-merror:0.041541
[6]	train-merror:0.038336	test-merror:0.038336
[7]	train-merror:0.035464	test-merror:0.035464
[8]	train-merror:0.033126	test-merror:0.033126
[9]	train-merror:0.030922	test-merror:0.030922
[10]	train-merror:0.028785	test-merror:0.028785
[11]	train-merror:0.026782	test-merror:0.026782
[12]	train-merror:0.025312	test-merror:0.025312
[13]	train-merror:0.024444	test-merror:0.024444
[14]	train-merror:0.022841	test-merror:0.022841
[15]	train-merror:0.021238	test-merror:0.021238
[16]	train-merror:0.020437	test-merror:0.020437
[17]	train-merror:0.019168	test-merror:0.019168

E:\anacoda\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\anacoda\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


============================ complete！ ====================================
################# ######目前正在跑第 2 个mall, 2/97##########################
[0]	train-merror:0.041859	test-merror:0.041859
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.032286	test-merror:0.032286
[2]	train-merror:0.02837	test-merror:0.02837
[3]	train-merror:0.025846	test-merror:0.025846
[4]	train-merror:0.023671	test-merror:0.023671
[5]	train-merror:0.0228	test-merror:0.0228
[6]	train-merror:0.021843	test-merror:0.021843
[7]	train-merror:0.021408	test-merror:0.021408
[8]	train-merror:0.02019	test-merror:0.02019
[9]	train-merror:0.018971	test-merror:0.018971
[10]	train-merror:0.01784	test-merror:0.01784
[11]	train-merror:0.016709	test-merror:0.016709
[12]	train-merror:0.016361	test-merror:0.016361
[13]	train-merror:0.016274	test-merror:0.016274
[14]	train-merror:0.015403	test-merror:0.015403
[15]	t

...

[25]	train-merror:0.012055	test-merror:0.012055
[26]	train-merror:0.01165	test-merror:0.01165
[27]	train-merror:0.011084	test-merror:0.011084
[28]	train-merror:0.01068	test-merror:0.01068
[29]	train-merror:0.010194	test-merror:0.010194
[30]	train-merror:0.009871	test-merror:0.009871
[31]	train-merror:0.009385	test-merror:0.009385
[32]	train-merror:0.008738	test-merror:0.008738
[33]	train-merror:0.008252	test-merror:0.008252
[34]	train-merror:0.007848	test-merror:0.007848
[35]	train-merror:0.007443	test-merror:0.007443
[36]	train-merror:0.006877	test-merror:0.006877
[37]	train-merror:0.006392	test-merror:0.006392
[38]	train-merror:0.005906	test-merror:0.005906
[39]	train-merror:0.005583	test-merror:0.005583
[40]	train-merror:0.005421	test-merror:0.005421
[41]	train-merror:0.005259	test-merror:0.005259
[42]	train-merror:0.005178	test-merror:0.005178
[43]	train-merror:0.004693	test-merror:0.004693
[44]	train-merror:0.004288	test-merror:0.004288
[45]	train-merror:0.004126	test-merror:0.004

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.084333	test-merror:0.084333
[2]	train-merror:0.077586	test-merror:0.077586
[3]	train-merror:0.064655	test-merror:0.064655
[4]	train-merror:0.052099	test-merror:0.052099
[5]	train-merror:0.047226	test-merror:0.047226
[6]	train-merror:0.043291	test-merror:0.043291
[7]	train-merror:0.039918	test-merror:0.039918
[8]	train-merror:0.036544	test-merror:0.036544
[9]	train-merror:0.033171	test-merror:0.033171
[10]	train-merror:0.029423	test-merror:0.029423
[11]	train-merror:0.026987	test-merror:0.026987
[12]	train-merror:0.024738	test-merror:0.024738
[13]	train-merror:0.023051	test-merror:0.023051
[14]	train-merror:0.022114	test-merror:0.022114
[15]	train-merror:0.021552	test-merror:0.021552
[16]	train-merror:0.018741	test-merror:0.018741
[17]	train-merror:0.016679	test-merror:0.016679
[18]	train-merror:0.016117	test-merror:0.016117
[19]	train-merror:0.015555	test-merror:0.015555
[20]	train-merror:0.01518	test-merror:

[40]	train-merror:0.005934	test-merror:0.005934
[41]	train-merror:0.005609	test-merror:0.005609
[42]	train-merror:0.005447	test-merror:0.005447
[43]	train-merror:0.005284	test-merror:0.005284
[44]	train-merror:0.004552	test-merror:0.004552
[45]	train-merror:0.004146	test-merror:0.004146
[46]	train-merror:0.003983	test-merror:0.003983
[47]	train-merror:0.003902	test-merror:0.003902
[48]	train-merror:0.003821	test-merror:0.003821
[49]	train-merror:0.00374	test-merror:0.00374
[50]	train-merror:0.003658	test-merror:0.003658
[51]	train-merror:0.003333	test-merror:0.003333
[52]	train-merror:0.003252	test-merror:0.003252
[53]	train-merror:0.003008	test-merror:0.003008
[54]	train-merror:0.002845	test-merror:0.002845
[55]	train-merror:0.00252	test-merror:0.00252
[56]	train-merror:0.002358	test-merror:0.002358
[57]	train-merror:0.002358	test-merror:0.002358
[58]	train-merror:0.002114	test-merror:0.002114
[59]	train-merror:0.00187	test-merror:0.00187
============================ complete！ =======

[14]	train-merror:0.022866	test-merror:0.022866
[15]	train-merror:0.021341	test-merror:0.021341
[16]	train-merror:0.019512	test-merror:0.019512
[17]	train-merror:0.017683	test-merror:0.017683
[18]	train-merror:0.017073	test-merror:0.017073
[19]	train-merror:0.015854	test-merror:0.015854
[20]	train-merror:0.015549	test-merror:0.015549
[21]	train-merror:0.014634	test-merror:0.014634
[22]	train-merror:0.013415	test-merror:0.013415
[23]	train-merror:0.012805	test-merror:0.012805
[24]	train-merror:0.0125	test-merror:0.0125
[25]	train-merror:0.011585	test-merror:0.011585
[26]	train-merror:0.01128	test-merror:0.01128
[27]	train-merror:0.010976	test-merror:0.010976
[28]	train-merror:0.009756	test-merror:0.009756
[29]	train-merror:0.009756	test-merror:0.009756
[30]	train-merror:0.009451	test-merror:0.009451
[31]	train-merror:0.008841	test-merror:0.008841
[32]	train-merror:0.008841	test-merror:0.008841
[33]	train-merror:0.008232	test-merror:0.008232
[34]	train-merror:0.007317	test-merror:0.00731

[54]	train-merror:0.013729	test-merror:0.013729
[55]	train-merror:0.013574	test-merror:0.013574
[56]	train-merror:0.013031	test-merror:0.013031
[57]	train-merror:0.012333	test-merror:0.012333
[58]	train-merror:0.01179	test-merror:0.01179
[59]	train-merror:0.011519	test-merror:0.011519
============================ complete！ ====================================
################# ######目前正在跑第 96 个mall, 96/97##########################
[0]	train-merror:0.065632	test-merror:0.065632
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.054392	test-merror:0.054392
[2]	train-merror:0.049977	test-merror:0.049977
[3]	train-merror:0.045795	test-merror:0.045795
[4]	train-merror:0.041948	test-merror:0.041948
[5]	train-merror:0.03877	test-merror:0.03877
[6]	train-merror:0.036094	test-merror:0.036094
[7]	train-merror:0.033585	test-merror:0.033585
[8]	train-merror:0.031612	test-merror:0.0316

In [29]:
subm = pd.read_csv("../result/sub.csv")

In [31]:
subm.shape

(501989, 2)

In [32]:
subm2 = pd.read_csv("../result/sub0.csv")

In [33]:
subm2.shape

(483931, 2)

In [34]:
subm.head()

,row_id,shop_id
0,118956,s_303701
1,119152,s_3040029
2,119161,s_383834
3,119201,s_873312
4,119214,s_643398


In [36]:
len(subm["row_id"].unique())

482005

In [37]:
subm = subm.drop_duplicates()

In [38]:
subm.shape

(482005, 2)

In [39]:
subm.to_csv("../result/sub.csv", index=False)

In [40]:
subm = pd.merge(subm, subm2, on=['row_id'])

,row_id,shop_id
0,118796,s_462041
1,118827,s_1966525
2,118946,s_8161
3,118980,s_1226168
4,119006,s_1083
